# Section 2.1

We chose to filter out rows with missing or invalid values for key fields.

This decision is based on guidance provided in the forum and the homework instructions, which state that if a row lacks the necessary information to determine viewing behavior or wealth characteristics, it can be safely disregarded.

In [0]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import *
from pyspark.sql.functions import sum as spark_sum, max as spark_max
from pyspark.sql.types import *
from functools import reduce

In [0]:
def load_csv_file(filename, schema):
  # Reads the relevant file from distributed file system using the given schema

  allowed_files = {'Daily program data': ('Daily program data', "|"),
                   'demographic': ('demographic', "|")}

  if filename not in allowed_files.keys():
    print(f'You were trying to access unknown file \"{filename}\". Only valid options are {allowed_files.keys()}')
    return None

  filepath = allowed_files[filename][0]
  dataPath = f"dbfs:/mnt/coursedata2024/fwm-stb-data/{filepath}"
  delimiter = allowed_files[filename][1]

  df = spark.read.format("csv")\
    .option("header","false")\
    .option("delimiter",delimiter)\
    .schema(schema)\
    .load(dataPath)
  return df
schemas_dict = {'Daily program data':
                  StructType([
                    StructField('prog_code', StringType()),
                    StructField('title', StringType()),
                    StructField('genre', StringType()),
                    StructField('air_date', StringType()),
                    StructField('air_time', StringType()),
                    StructField('Duration', FloatType())
                  ]),
                'viewing':
                  StructType([
                    StructField('device_id', StringType()),
                    StructField('event_date', StringType()),
                    StructField('event_time', IntegerType()),
                    StructField('mso_code', StringType()),
                    StructField('prog_code', StringType()),
                    StructField('station_num', StringType())
                  ]),
                'viewing_full':
                  StructType([
                    StructField('mso_code', StringType()),
                    StructField('device_id', StringType()),
                    StructField('event_date', IntegerType()),
                    StructField('event_time', IntegerType()),
                    StructField('station_num', StringType()),
                    StructField('prog_code', StringType())
                  ]),
                'demographic':
                  StructType([StructField('household_id',StringType()),
                    StructField('household_size',IntegerType()),
                    StructField('num_adults',IntegerType()),
                    StructField('num_generations',IntegerType()),
                    StructField('adult_range',StringType()),
                    StructField('marital_status',StringType()),
                    StructField('race_code',StringType()),
                    StructField('presence_children',StringType()),
                    StructField('num_children',IntegerType()),
                    StructField('age_children',StringType()), #format like range - 'bitwise'
                    StructField('age_range_children',StringType()),
                    StructField('dwelling_type',StringType()),
                    StructField('home_owner_status',StringType()),
                    StructField('length_residence',IntegerType()),
                    StructField('home_market_value',StringType()),
                    StructField('num_vehicles',IntegerType()),
                    StructField('vehicle_make',StringType()),
                    StructField('vehicle_model',StringType()),
                    StructField('vehicle_year',IntegerType()),
                    StructField('net_worth',IntegerType()),
                    StructField('income',StringType()),
                    StructField('gender_individual',StringType()),
                    StructField('age_individual',IntegerType()),
                    StructField('education_highest',StringType()),
                    StructField('occupation_highest',StringType()),
                    StructField('education_1',StringType()),
                    StructField('occupation_1',StringType()),
                    StructField('age_2',IntegerType()),
                    StructField('education_2',StringType()),
                    StructField('occupation_2',StringType()),
                    StructField('age_3',IntegerType()),
                    StructField('education_3',StringType()),
                    StructField('occupation_3',StringType()),
                    StructField('age_4',IntegerType()),
                    StructField('education_4',StringType()),
                    StructField('occupation_4',StringType()),
                    StructField('age_5',IntegerType()),
                    StructField('education_5',StringType()),
                    StructField('occupation_5',StringType()),
                    StructField('polit_party_regist',StringType()),
                    StructField('polit_party_input',StringType()),
                    StructField('household_clusters',StringType()),
                    StructField('insurance_groups',StringType()),
                    StructField('financial_groups',StringType()),
                    StructField('green_living',StringType())
                  ])
}

In [0]:
# Load datasets using pre-defined schema and helper
program_df = load_csv_file('Daily program data', schemas_dict['Daily program data'])
demographic_df = load_csv_file('demographic', schemas_dict['demographic'])

# Load reference data (parquet)
ref_data_schema = StructType([
    StructField('device_id', StringType()),
    StructField('dma', StringType()),
    StructField('dma_code', StringType()),
    StructField('household_id', IntegerType()),
    StructField('zipcode', IntegerType())
])
reference_df = spark.read.format('parquet') \
    .option("inferSchema", "true") \
    .load("dbfs:/FileStore/ddm/ref_data")

# Load viewing data (CSV with schema)
viewing_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("delimiter", ",") \
    .schema(schemas_dict['viewing_full']) \
    .load("dbfs:/FileStore/ddm/10m_viewing")

In [0]:
viewing_df.dropDuplicates()
program_df.dropDuplicates()
demographic_df.dropDuplicates()
reference_df.dropDuplicates()


viewing_df = viewing_df.filter(col("prog_code").isNotNull() &
                               col("device_id").isNotNull())

program_df = program_df.filter(col("title").isNotNull() &
                               col("genre").isNotNull() &
                               col("prog_code").isNotNull()).dropDuplicates(["prog_code", "genre"])

demographic_df = demographic_df.filter(col("household_id").isNotNull()).dropDuplicates(["household_id"])


reference_df = reference_df.filter(col("household_id").isNotNull() &
                                   col("DMA").isNotNull() &
                                   (col("DMA") != "Unknown"))

reference_df = reference_df.withColumn("household_id",
    lpad(col("household_id").cast("string"), 8, "0")  # Pad with zeros to 8 characters
)

demographic_df = demographic_df.withColumn("num_adults", col("num_adults").cast("int")) \
    .withColumn("income", 
                when(col("income").rlike("^[A-D]$"), 
                     ascii(substring(col("income"), 1, 1)) - ascii(lit("A")) + 10)
                .otherwise(col("income").cast("int"))
    ).withColumn("household_id",
                lpad(col("household_id").cast("string"), 8, "0")  # Pad with zeros to 8 characters
)

**Genres:**

In [0]:
# STEP 1: Join viewing_df with cleaned program_df to get genre
view_prog_df = viewing_df.join(program_df.select("prog_code", "genre"), on="prog_code")

# STEP 2: Join with reference_df to get household_id
view_prog_ref_df = view_prog_df.join(reference_df.select("device_id", "household_id"), on="device_id")

# STEP 3: Join with demographic_df (broadcasted for performance)
view_prog_ref_demo_df = view_prog_ref_df.join(
    broadcast(demographic_df.select("household_id", "household_size")),
    on="household_id"
)

# STEP 4: Explode comma-separated genres into separate rows
exploded_genres_df = view_prog_ref_demo_df.withColumn("genre", explode(split(col("genre"), ",\\s*")))

# STEP 5: Drop duplicates (one row per household per genre)
unique_viewers = exploded_genres_df \
    .filter(col("genre").isNotNull() & (col("genre") != "")) \
    .select("household_id", "genre", "household_size") \
    .dropDuplicates()

# STEP 6: Aggregate total household size per genre
genre_agg_df = (
    unique_viewers
    .groupBy("genre")
    .agg(spark_sum("household_size").alias("total_viewers"))
    .orderBy(col("total_viewers").desc())
)
# STEP 7: Show top 5 genres
top5_genres = genre_agg_df.limit(5)
top5_genres.cache()
top5_genres.show()

# STEP 8: Show total viewers across top 5
total_viewers_top5 = top5_genres.agg(spark_sum("total_viewers")).collect()[0][0]
print("Total viewers for top 5 genres:", total_viewers_top5)

+-------+-------------+
|  genre|total_viewers|
+-------+-------------+
|   News|       615298|
|Reality|       610470|
|   Talk|       537717|
| Comedy|       509668|
| Sitcom|       502748|
+-------+-------------+

Total viewers for top 5 genres: 2775901


**DMAs**

In [0]:
# Count devices per DMA using only reference data
dma_device_counts = (
    reference_df.groupBy("DMA")
    .agg(count("device_id").alias("device_count"))
    .orderBy(col("device_count").desc())
)

# Get top 5 DMAs
top5_dmas = dma_device_counts.limit(5)

# Next, Get total people in top 5 DMAs
top5_dma_names = [row["DMA"] for row in top5_dmas.collect()]

dma_people_df = reference_df.join(demographic_df.select("household_id", "household_size"), on="household_id")
dma_people_filtered = dma_people_df.filter(col("DMA").isin(top5_dma_names))

dma_totals = (
    dma_people_filtered
    .select("DMA", "household_id", "household_size")
    .dropDuplicates()
    .groupBy("DMA")
    .agg(spark_sum("household_size").alias("total_people"))
)

dma_totals.cache()
dma_totals.show(truncate=False)

total_people_top5_dmas = dma_totals.agg(spark_sum("total_people")).collect()[0][0]
print("Total people in top 5 DMAs:", total_people_top5_dmas)

+--------------------------+------------+
|DMA                       |total_people|
+--------------------------+------------+
|Little Rock-Pine Bluff    |31652       |
|Seattle-Tacoma            |35124       |
|Toledo                    |24108       |
|Wilkes Barre-Scranton-Hztn|42844       |
|Charleston-Huntington     |60656       |
+--------------------------+------------+

Total people in top 5 DMAs: 194384


**Program Titles**

In [0]:
# Step 1: Filter early and reduce joins
demographic_kids_df = demographic_df.filter(upper(col("presence_children")) == "Y").select("household_id", "household_size")

viewing_with_household_df = viewing_df.join(broadcast(reference_df.select("device_id", "household_id")), on="device_id")
viewing_kids_df = viewing_with_household_df.join(broadcast(demographic_kids_df), on="household_id")

viewing_kids_prog_df = viewing_kids_df.join(broadcast(program_df.select("prog_code", "title")), on="prog_code")

# Step 2: Drop partial duplicates (not full row)
unique_views = viewing_kids_prog_df.select("household_id", "title", "household_size").dropDuplicates(["household_id", "title"])

# Step 3: Repartition to avoid skew, then aggregate
unique_views = unique_views.repartition("title")

top_kids_programs = (
    unique_views
    .groupBy("title")
    .agg(spark_sum("household_size").alias("total_viewers"))
    .orderBy(col("total_viewers").desc())
    .limit(5)
)

top_kids_programs.cache()
top_kids_programs.show()

# Step 4: Sum
total_viewers_top5_kids = top_kids_programs.agg(spark_sum("total_viewers")).collect()[0][0]
print("Total viewers in top 5 kids programs:", total_viewers_top5_kids)


+-------------------+-------------+
|              title|total_viewers|
+-------------------+-------------+
| College Basketball|        74693|
|   Paid Programming|        73049|
|       SportsCenter|        58833|
|The Big Bang Theory|        53317|
|              Today|        44178|
+-------------------+-------------+

Total viewers in top 5 kids programs: 304070


# Section 2.2

**Wealth Score per DMA**

In [0]:
demographic_df = demographic_df.filter(col("net_worth").isNotNull() &
                                       col("income").isNotNull())

In [0]:
# Join reference with demographic to get DMA, net worth, and income
dma_wealth_df = reference_df.select("household_id", "DMA").join(
    demographic_df.select("household_id", "net_worth", "income"),
    on="household_id"
)

# Compute average net worth and income per DMA
dma_avg_df = dma_wealth_df.groupBy("DMA").agg(
    avg("net_worth").alias("avg_net_worth"),
    avg("income").alias("avg_income")
)

max_income = int(dma_wealth_df.agg(spark_max("income")).collect()[0][0])
max_net_worth = int(dma_wealth_df.agg(spark_max("net_worth")).collect()[0][0])

# Compute wealth score and sort
dma_score_df = dma_avg_df.withColumn(
    "wealth_score",
    (col("avg_net_worth") / max_net_worth) + (col("avg_income") / max_income)
).orderBy(col("wealth_score").desc())

# Select top 10 DMAs by wealth score
top10_dmas = dma_score_df.limit(10).select("DMA", "wealth_score")

**Genre Popularity per DMA, Then Rank Genres by Popularity Within Each DMA**

In [0]:
# Replace the join in Step 1 with this:
view_prog_df = viewing_df.join(program_df.select("prog_code", "genre"), on="prog_code", how="inner")

view_prog_ref_df = view_prog_df.join(
    reference_df.select("device_id", "DMA", "household_id"),
    on="device_id",
    how="inner"
)

# Step 2: Explode genre list into individual genres
genre_popularity_df = (
    view_prog_ref_df
    .withColumn("single_genre", explode(split(col("genre"), ",\\s*")))
    .select("DMA", "device_id", "single_genre")
    .dropDuplicates(["device_id", "single_genre"])  # avoid double-counting devices watching the same genre
    .groupBy("DMA", "single_genre")
    .agg(countDistinct("device_id").alias("device_count"))
)
# Step 3: Join with top 10 DMAs
top_dma_genres = top10_dmas.join(genre_popularity_df, on="DMA", how="left")

# Step 4: Rank genres by total household size within each DMA
window_spec = Window.partitionBy("DMA").orderBy(col("device_count").desc())

ranked_genres = top_dma_genres.withColumn("rank", row_number().over(window_spec))

# Sort genres by rank within each DMA, and collect (rank, genre) structs
ranked_lists = (
    ranked_genres
    .select("DMA", "wealth_score", "rank", "single_genre")
    .orderBy("DMA", "rank")
    .groupBy("DMA", "wealth_score")
    .agg(collect_list(struct("rank", "single_genre")).alias("ranked_genre_structs"))
)

**Assign 11 Unique Genres per DMA (No Repeats Across DMAs)**

In [0]:
top_dma_genres_list = sorted(ranked_lists.collect(), key=lambda x: -x["wealth_score"])  # Sort by wealth descending

used_genres = set()
final_output = []

for row in top_dma_genres_list:
    dma = row["DMA"]
    score = row["wealth_score"]
    genre_structs = row["ranked_genre_structs"]

    # Sort by rank just in case
    genre_structs = sorted(genre_structs, key=lambda x: x["rank"])

    selected = []
    for g in genre_structs:
        genre = g["single_genre"]
        if genre and genre not in used_genres:
            selected.append(genre)
            used_genres.add(genre)
        if len(selected) == 11:
            break

    final_output.append((dma, score, selected))

# Convert to DataFrame
final_schema = ["DMA", "Wealth_Score", "Ordered_List_Of_Genres"]
final_result_df = spark.createDataFrame(final_output, final_schema)

final_result_df.orderBy(col("Wealth_Score").desc()).show(truncate=False)


+--------------------------+------------------+-----------------------------------------------------------------------------------------------------------------------------+
|DMA                       |Wealth_Score      |Ordered_List_Of_Genres                                                                                                       |
+--------------------------+------------------+-----------------------------------------------------------------------------------------------------------------------------+
|San Antonio               |1.623931623931624 |[]                                                                                                                           |
|San Francisco-Oak-San Jose|1.5422277562565332|[Reality, News, Comedy, Music, Sitcom, Talk, Drama, Documentary, Adventure, Children, Action]                                |
|Baltimore                 |1.5220570915273188|[]                                                                                 